In [4]:
#import stuff

# NOTE: gutenberg MUST be run in a separate notebook, collab cannot handle gutenberg's necessary python 2.7 dependencies, unlike Jupyter
# ^^ for more, see: https://pypi.org/project/Gutenberg/  we cannot run the requisite "brew install berkeley-db4" command 


#TensorFlow
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

#SKLearn
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF



#Other
# from gutenberg.acquire import load_etext
# from gutenberg.cleanup import strip_headers
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
import re

In [3]:
#gutenberg is used to fetch book txt files, so long as we give it the book ID
#we will, in addition to that, feed in the # of lines we want to skip (think of it like skipping the first 10 pages),
#and give it a personal name

books = [
#   BookID, skip N lines
#   (26715, 1000, 'Victorian songs'),
#   (30235, 580, 'Baldwin collection'),
#   (35402, 710, 'Swinburne collection'),
#   (574, 15, 'Blake'),
#   (1304, 172, 'Bulchevys collection'),
#   (19221, 223, 'Palgrave-Pearse collection'),
#   (15553, 522, 'Knowles collection') ,
    (17650, 6605, 'The Sonnets, Triumphs, and Other Poems of Petrarch by Francesco Petrarca') 
#                                    Link: https://www.gutenberg.org/ebooks/17650
#                                    6605 is the # of lines that we skip to get 
#                                    to the poems.
]

# Get cleaned text file

In [ ]:
#Save as raw.txt file
with open('data/poetry/raw.txt', 'w') as ofp:
  lineno = 0
  for (id_nr, toskip, title) in books:
    startline = lineno
    text = strip_headers(load_etext(id_nr, mirror='http://mirrors.xmission.com/gutenberg/')).strip()
    lines = text.split('\n')[toskip:]
    for line in lines:
      if (len(line) > 0 
          and line.upper() != line 
          and not re.match('.*[0-9]+.*', line)
          and len(line) < 50
         ): #skip Titles
        cleaned = re.sub('[^a-z\'\-]+', ' ', line.strip().lower()) #all lowercase, only letters
        ofp.write(cleaned)
        ofp.write('\n')
        lineno = lineno + 1 #next line
      else:
        ofp.write('\n')
    print('Wrote lines {} to {} from {}'.format(startline, lineno, title))

In [5]:
#Connect to Google drive, where raw.txt is stored. 
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
#Now that we are connected to our gDrive, we can read in raw.txt
raw = pd.read_csv('/content/drive/My Drive/notebooks/raw.txt')
raw = raw.values.tolist()

print("Example data structure, note that it is a list of lists: \n",raw[0:2])

Example data structure, note that it is a list of lists: 
 [['of those sad sighs with which my heart i fed'], ['when early youth my mazy wanderings led ']]


# LDA/NMF

In [98]:
#Define corpus as just one long list as opposed to list of lists?
corpus = [','.join(x) for x in raw]

In [90]:
#Instantiate count vectorizer
CV = CountVectorizer(ngram_range=(1, 1), stop_words='english')

#Fit to data
Vector = CV.fit_transform(corpus)

#LDA
lda = LatentDirichletAllocation(n_components=5)
lda_Fit = lda.fit_transform(Vector)

#NMF
#Look @ top 10 words, name as clusters - ask the user to pick one of those
nmf = NMF(n_components=100, init='random', random_state=0)
nmf_Fit = nmf.fit_transform(Vector)
nmf_Components = nmf.components_


print(Vector.shape[0],'poems analyzed by CountVectorizer')
print(nmf_Components.shape[1], 'poems analyzed by NMF')
print(len(lda_Fit),'poems analyzed by LDA')

10982 poems analyzed by CountVectorizer
8217 poems analyzed by NMF
10982 poems analyzed by LDA


In [80]:
nmf_Components.shape

(50, 8217)

In [91]:
#df is FIT ON NMF
#df = pd.DataFrame(data = Vector.toarray(), columns=CV.get_feature_names())
df = pd.DataFrame(data = nmf_Components, columns=CV.get_feature_names())

## Analyze NMF/LDA

In [12]:
features = CV.get_feature_names()

#Words from NMF
a = nmf_Components.argsort(axis=1)[:,-1:-7:-1]
nmf_words = [[features[e] for e in l] for l in a]
print('NMF poem topic words \n', nmf_words)

#Words from LDA
b = lda.components_.argsort(axis=1)[:,-1:-7:-1]
lda_words = [[corpus[e] for e in l] for l in b]
print('LDA poem topic words \n', lda_words)

NMF poem topic words 
 [['love', 'long', 'cruel', 'death', 'fortune', 'mind'], ['heart', 'thy', 'life', 'er', 'day', 'eyes']]
LDA poem topic words 
 [['which gilds through life the gloomy hour ', 'i said as far from men ', 'centre of honour perfect and alone ', 'has lust his death from supreme beauty met ', 'when hope was in its bloomy prime ', 'that graced her age unable to depart '], ['the bitter and the sweet on which i feed ', 'in arrowy jets of fire thence came and went', 'which gilds through life the gloomy hour ', "e'en its fair promise--and with fatal pride ", "my eyes enamour'd of their grief intense ", 'sweet keys of my gone heart '], ['eternal tears my stricken heart overflow ', 'the bitter and the sweet on which i feed ', ' n cos bello il sol giammai levarsi ', "what beast within my fold has leap'd to feed ", "ah to thy friend too well may be address'd ", ' dolci ire dolci sdegni e dolci paci '], ['blest the sweet pain of which i was the prey ', 'the bitter and the sweet on

# KNN Algorithm

In [ ]:
def run_KMeans(max_k, data):
    max_k += 1
    kmeans_results = dict()
    for k in range(2 , max_k):
        kmeans = KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 10
                               , tol = 0.0001
                               , n_jobs = -1
                               , random_state = 1
                               , algorithm = 'full')

        kmeans_results.update( {k : kmeans.fit(data)} )
        
    return kmeans_results

# Running Kmeans
k = 8
kmeans_results = run_KMeans(k, df)

In [ ]:
# Run the Kmeans algorithm and get the index of data points clusters
sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(X_std)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [92]:
# Running Kmeans
k = 50
kmeans = KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 10
                               , tol = 0.001
                               , n_jobs = -1
                               , random_state = 1
                               , algorithm = 'full').fit(df)

## KMeans Analysis

In [86]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    print(labels)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        print(id_temp)
        x_means = np.mean(df_arr[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = CV.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        print(best_features)
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
df_arr = df.to_numpy()
prediction = kmeans.predict(df)
n_feats = 20
dfs = get_top_features_cluster(df_arr, prediction, n_feats)
plotWords(dfs, 13)

Using word2vec -- what words are symbols of femininity? 

In [58]:
from gensim.models import Word2Vec

In [ ]:
df = pd.read_csv('/content/drive/My Drive/notebooks/raw.txt')
df = df.values.tolist()

model = Word2Vec(df, min_count=1,size=32)

In [63]:
df

[['of those sad sighs with which my heart i fed'],
 ['when early youth my mazy wanderings led '],
 ['fondly diverse from what i now appear '],
 ['from those by whom my various style is read '],
 ['not only pardon but perhaps a tear '],
 ['but now i clearly see that of mankind'],
 ['and self-reproach with frequent blushes teem '],
 ['while of my frenzy shame the fruit i find '],
 ["that the world's joy is but a flitting dream "],
 ["o ye who list in scatter'd verse the sound"],
 ['of all those sighs with which my heart i fed '],
 ['when i by youthful error first misled '],
 ['unlike my present self in heart was found '],
 ["if e'er true love its influence o'er ye shed "],
 ["oh let your pity be with pardon crown'd "],
 ['but now full well i see how to the crowd'],
 ['for length of time i proved a public jest '],
 ["e'en by myself my folly is allow'd "],
 ['and of my vanity the fruit is shame '],
 ['repentance and a knowledge strong imprest '],
 ['that worldly pleasure is a passing dream

In [72]:
# #Instantiate count vectorizer
# CV = CountVectorizer(ngram_range=(1, 1), stop_words='english')

# #Fit to data
# Vector = CV.fit_transform(corpus)

# #NMF
# #Look @ top 10 words, name as clusters - ask the user to pick one of those
# nmf = NMF(n_components=100, init='random', random_state=0)
# nmf_Fit = nmf.fit_transform(Vector)
# nmf_Components = nmf.components_

from keras.preprocessing.text import Tokenizer
t = Tokenizer(num_words=10) 
corpus = [','.join(x) for x in raw]
print(corpus)
tokenized = t.fit_on_texts(corpus)
tokenized.word_index

['of those sad sighs with which my heart i fed', 'when early youth my mazy wanderings led ', 'fondly diverse from what i now appear ', 'from those by whom my various style is read ', 'not only pardon but perhaps a tear ', 'but now i clearly see that of mankind', 'and self-reproach with frequent blushes teem ', 'while of my frenzy shame the fruit i find ', "that the world's joy is but a flitting dream ", "o ye who list in scatter'd verse the sound", 'of all those sighs with which my heart i fed ', 'when i by youthful error first misled ', 'unlike my present self in heart was found ', "if e'er true love its influence o'er ye shed ", "oh let your pity be with pardon crown'd ", 'but now full well i see how to the crowd', 'for length of time i proved a public jest ', "e'en by myself my folly is allow'd ", 'and of my vanity the fruit is shame ', 'repentance and a knowledge strong imprest ', 'that worldly pleasure is a passing dream ', 'ye who may listen to each idle strain', "in life's first

AttributeError: ignored

In [49]:
model = gensim.models.Word2Vec(tokenized,min_count=1,size=32)

print(model['those'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: ignored

Rules of Petrarchan sonnet
- 14 lines
- 8 line subsection / 6 line subsection (seste)
- 8 line --> ABBA ABBA 
- 6 line --> CDE CDE 


In [ ]:
with open('data/poetry/raw.txt', 'r') as rawfp,\
 open('data/poetry/input.txt', 'w') as infp,\
 open('data/poetry/output.txt', 'w') as outfp:
   
   prev_line = ''
   for curr_line in rawfp:
       curr_line = curr_line.strip()
       # poems break at empty lines, so this ensures we train only
       # on lines of the same poem
       if len(prev_line) > 0 and len(curr_line) > 0:       
           infp.write(prev_line + '\n')
           outfp.write(curr_line + '\n')
       prev_line = curr_line